In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense 
from tensorflow.keras.callbacks import TensorBoard

In [3]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils


In [4]:
def mediapipe_detection(image, model):
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
            image.flags.writeable = False 
            results = model.process(image)   
            image.flags.writeable = True    
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
            return image, results

In [5]:
def draw_landmarks(image, results):
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)  
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)


In [6]:
def landmark_style(image, results):
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=1)
                                )  
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=1)
                                ) 
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=1)
                                )  

In [7]:
print(mp_holistic.POSE_CONNECTIONS)

frozenset({(15, 21), (16, 20), (18, 20), (3, 7), (14, 16), (23, 25), (28, 30), (11, 23), (27, 31), (6, 8), (15, 17), (24, 26), (16, 22), (4, 5), (5, 6), (29, 31), (12, 24), (23, 24), (0, 1), (9, 10), (1, 2), (0, 4), (11, 13), (30, 32), (28, 32), (15, 19), (16, 18), (25, 27), (26, 28), (12, 14), (17, 19), (2, 3), (11, 12), (27, 29), (13, 15)})


In [8]:
def extract_keypoints(result):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


In [9]:
DATA_PATH = os.path.join('FinalDataSet')
actions = np.array(['bye', 'hello', 'help', 'iamsleepy', 'iloveyou', 'shocked', 'thanks'])
sequence_length = 30

In [10]:
def get_no_of_images(folder_name):
    DATA_PATH = "D:\Sign _Language_ Detection\Production-Project-\FinalDataSet"
    import os
    no_of_images = os.listdir(os.path.join(DATA_PATH, folder_name))
    return len(no_of_images)

In [11]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.load_weights('D:\Sign _Language_ Detection\Production-Project-\May10_Second.h5')

In [12]:
colors = [(0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0), (0,0,0),(0,0,0)]

In [13]:
def detect_motion(previous_frame,prepared_frame):
    
    # calculate difference and update previous frame
    diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)
    previous_frame = prepared_frame

    # 4. Dilute the image a bit to make differences more seeable; more suitable for contour detection
    kernel = np.ones((5, 5))
    diff_frame = cv2.dilate(diff_frame, kernel, 1)

    print(diff_frame)
    
#     # 5. Only take different areas that are different enough (>20 / 255)
#     thresh_frame = cv2.threshold(src=diff_frame, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1]
#     print("hahahahah", thresh_frame)
    

In [14]:
def prob_viz(res, actions, input_frame, colors, motion):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        if motion:
            cv2.rectangle(output_frame, (0,60+num*40), (int(prob*200), 90+num*40), colors[num], -1)
            cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

In [15]:
import cv2
sequence = []
sentence = []
threshold = 0.4

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
cap = cv2.VideoCapture(0)
_, prev_frame = cap.read()
count = 1
motion_detected = False
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
        _, frame_diff_threshold = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
        frame_diff_threshold = cv2.dilate(frame_diff_threshold, None, iterations=2)
        contours, _ = cv2.findContours(frame_diff_threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        count+=1
        if count >= 33:
            count = 1
            motion_detected = False
        
        for contour in contours:
            if cv2.contourArea(contour) > 9000: 
                motion_detected = True
        prev_frame = frame
        
        
        image, results = mediapipe_detection(frame, holistic)
        landmark_style(image, results)
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            if res[np.argmax(res)] > threshold: 
                image = prob_viz(res, actions, image, colors,motion_detected)

            cv2.imshow('Gesture Recognition', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 67ms/step


1/1 [==============================] - 0s 58ms/step


1/1 [==============================] - 0s 55ms/step


1/1 [==============================] - 0s 58ms/step
